# Logistic Regression

In [1]:
import math
import numpy as np
import pandas as pd
import sklearn.linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Simulierte Daten

In [27]:
def generate_prices(observations_count, prices_range):
    return np.random.choice(prices_range, observations_count)

def generate_competitor_prices(observations_count, competitors_count, prices_range):
    return np.matrix([generate_prices(observations_count, prices_range) for i in range(0, competitors_count)])
    
def calculate_ranks(observations_count, competitors_count, competitor_prices, prices):
    return [1 + len([1 for j in range(competitors_count) if competitor_prices[j,i] < prices[i]])
            for i in range(observations_count)]

def calculate_sold_probs_A(observations_count, ranks, competitors_count, prices):
    max_prob = lambda i: 1 - ((0.3 * ranks[0][i])/(competitors_count[0] + 1)) - 0.05 * prices[0][i] + (-0.0125 * (prices[0][i] - prices[1][i]) + 0.25)
    return [np.maximum(0, np.round(np.random.uniform(0, max_prob(i)))) for i in range(observations_count)]

def calculate_sold_probs_B(observations_count, ranks, competitors_count, prices):
    max_prob = lambda i: 1 - ((0.3 * ranks[1][i])/(competitors_count[1] + 1)) - 0.05 * prices[1][i] + (0.0125 * (prices[0][i] - prices[1][i]) + 0.25)
    return [np.maximum(0, np.round(np.random.uniform(0, max_prob(i)))) for i in range(observations_count)]

observations_count = 1000
prices_ranges = [np.arange(1, 20.1, 0.1) for i in range(2)] #TODO adopt prob function if price changes
prices = [generate_prices(observations_count, prices_ranges[i]) for i in range(2)] # TODO p1 > p2?
competitors_count = [5, 5]
competitor_prices = [generate_competitor_prices(observations_count, competitors_count[i], prices_ranges[i]) for i in range(2)]
ranks = [calculate_ranks(observations_count, competitors_count[i], competitor_prices[i], prices[i]) for i in range(2)]
sold = [calculate_sold_probs_A(observations_count, ranks, competitors_count, prices),
        calculate_sold_probs_B(observations_count, ranks, competitors_count, prices)]

## Regression

In [28]:
def get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices):
    explanatory_1 = [1 for j in range(observations_count)]
    explanatory_2 = [ranks[0][j] for j in range(observations_count)]
    explanatory_3 = [prices[0][j] - np.min([competitor_prices[0][k,j] for k in range(0, competitors_count[0])]) for j in range(observations_count)]
    return np.matrix([explanatory_1, explanatory_2, explanatory_3])

def get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices):
    explanatory_1 = [1 for j in range(observations_count)]
    explanatory_2 = [ranks[1][j] for j in range(observations_count)]
    explanatory_3 = [prices[1][j] - np.min([competitor_prices[1][k,j] for k in range(0, competitors_count[1])]) for j in range(observations_count)]
    return np.matrix([explanatory_1, explanatory_2, explanatory_3])

explanatory_vars = [get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices),
                    get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices)]
logits = [sm.Logit(sold[i], explanatory_vars[i].transpose()) for i in range(2)]
results = [logits[i].fit() for i in range(2)]
beta = [results[i].params for i in range(2)]
beta

Optimization terminated successfully.
         Current function value: 0.375414
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.377487
         Iterations 7


[array([ 0.42237045, -0.28880732, -0.19256854]),
 array([ 0.35702097, -0.3539194 , -0.16331534])]

In [32]:
[results[i].aic for i in range(2)]

[756.82862886997259, 760.97389058498811]

## Optimierung

In [55]:
def get_price_index(price):
    return int(price / 0.1 - 10) # TODO dependent on price range?

def calculate_sale_probs(beta, explanatory_vars, prices_range):
    L = lambda price: np.sum([beta[m] * explanatory_vars[m,get_price_index(price)] for m in range(explanatory_vars.shape[0])])
    return [np.exp(L(price)) / (1 + np.exp(L(price))) for price in prices_range]

def bellman(n, sale_probs, prices_range, holding_cost_rate, delta, values, step):
    prob_A = lambda price: np.sum([sale_probs[0, i, get_price_index(price)] for i in range(2)])
    prob_B = lambda price: np.sum([sale_probs[1, i, get_price_index(price)] for i in range(2)])
    todays_profit = lambda prices: np.sum([np.min([n[j], i]) * prices[j] - n[j] * holding_cost_rate[j] for (i, j) in (range(2), range(2))])
    disc_exp_fut_profits = np.sum([delta * values[step + 1, np.max([0, n[0]-i]), np.max([0, n[1]-i])] for i in range(2)])
    
    bellman_for_combinations = []
    for price_A in prices_range[0]:
        for price_B in prices_range[1]:
            bellman_for_combinations.append(prob_A(price_A) * prob_B(price_B) * (todays_profit([price_A, price_B]) + disc_exp_fut_profits))
    return np.max(bellman_for_combinations)

competitor_prices = [generate_competitor_prices(observations_count, competitors_count[i], prices_ranges[i]) for i in range(2)]
explanatory_vars = [get_explanatory_vars_A(observations_count, competitors_count, ranks, competitor_prices),
                    get_explanatory_vars_B(observations_count, competitors_count, ranks, competitor_prices)]

p = [calculate_sale_probs(beta[i], explanatory_vars[i], prices_range[i]) for i in range(2)]
sale_probs = np.array([[[1 - j for j in p[i]], p[i]] for i in range(2)])

delta = 0.99
holding_cost_rate = [0.01, 0.01]
init_inventory = [3, 3]
steps = 10
values = np.empty(shape=(steps + 1, init_inventory[0], init_inventory[1]))
for step in range(steps, -1, -1):
    for n_A in range(init_inventory[0]):
        for n_B in range(init_inventory[1]):
            if step == steps or n_A == 0 or n_B == 0:
                values[step, n_A, n_B] = 0
            else:
                values[step, n_A, n_B] = bellman([n_A, n_B], sale_probs, prices_range, holding_cost_rate, delta, values, step)

"""
opt_prices = np.empty(init_inventory)
for n in range(init_inventory):
    for price in prices_range:
        if np.sum([sale_prob[i,get_price_index(price)]*(np.min([n,i]) * price - n * holding_cost_rate + delta * values[1, np.max([0, n-i])]) for i in range(2)]) == values[0, n]:
            opt_prices[n] = price
opt_values = [values[0,n] for n in range(init_inventory)]
opt_prices
"""
values

array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.19123585, -0.3824717 ],
        [ 0.        , -0.19123585, -1.22717935]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.17296551, -0.34593101],
        [ 0.        , -0.17296551, -1.02620555]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.15451061, -0.30902122],
        [ 0.        , -0.15451061, -0.84165661]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.1358693 , -0.27173861],
        [ 0.        , -0.1358693 , -0.67388485]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.1170397 , -0.2340794 ],
        [ 0.        , -0.1170397 , -0.52324803]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.0980199 , -0.1960398 ],
        [ 0.        , -0.0980199 , -0.39010942]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        , -0.07880798, -0.15761596],
        